# Title
[]()

In [1]:
import requests
import json
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
import pickle
import os

In [330]:
with open("credentials.json") as f:
    credentials = json.load(f)
# user_id = os.environ['fb_user_id']

In [4]:
def get_user_ig_post(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption%2Ccomments_count%2Cis_shared_to_feed%2Clike_count%2Cmedia_type%2Cmedia_url%2Cmedia_product_type%2Cpermalink%2Ctimestamp%2Ccomments&access_token='
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption,comments_count,is_shared_to_feed,like_count,media_type,media_url,media_product_type,permalink,timestamp,comments&access_token='
    url_without_token = f'{url_root}{ig_user_id}/media?access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df_test, response_json_test = get_user_ig_post(credentials['ig_user_id'], 
        credentials['access_token'], pages=2, filename='my_ig_posts_2p_2023-01-23')


https://graph.facebook.com/v15.0/17841403231458630/media?access_token=
Response status code:  200
Response status code:  200
Number of posts: 50
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/my_ig_posts_2p_2023-01-23.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/my_ig_posts_2p_2023-01-23.sav


In [13]:
df_test_am, response_json_test_am = get_user_ig_post(credentials['am_ig_user_id'], 
        credentials['am2_access_token'], pages=2, filename='am_ig_posts_2p_2023-01-25')


https://graph.facebook.com/v15.0/17841401256046961/media?access_token=
Response status code:  200
Response status code:  200
Number of posts: 50
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/am_ig_posts_2p_2023-01-25.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/am_ig_posts_2p_2023-01-25.sav


In [104]:
df_test_am.columns

Index(['caption', 'like_count', 'comments_count', 'media_type',
       'media_product_type', 'media_url', 'permalink', 'timestamp', 'id',
       'comments.data', 'thumbnail_url', 'comments.paging.cursors.after',
       'comments.paging.next'],
      dtype='object')

# `get_user_ig_post_text`

In [141]:
# SH 2023-01-26 0:29 See updated version
def get_user_ig_post_text(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ SH 2023-01-25 21:52
    Pull the media from a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - pages: Number of pages of results to retrieve starting in reverse chronological order.
            Each page of results provides 25 posts.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information (1 post/reel per row)
            - caption
            - like count
            - comments count
            - top-level comments: If the amount of comments is high (>53 commments), it will provide the endpoints, 
                it will return additional endpoints for additional comments ('comments.paging.*').
            - media type: video or image
            - media product type: reels or feed
            - media URL (not available for all reels)
            - permalink
            - timestamp
            - post id
            - comments.data (list of dictionaries): timestamp, username, text, like count, and comment id.
            - thumbnail URL of videos
        - response_json: JSON object with each page number of results as the key (starting with 1)

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Ctimestamp%2Ccomments%7Btimestamp%2Cusername%2Ctext%2Clike_count%7D%2Cthumbnail_url&access_token='
    url_without_token = f'{url_root}{ig_user_id}/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [107]:


df_test_am, response_json_test_am = get_user_ig_post_text(credentials['am_ig_user_id'], 
        credentials['am2_access_token'], pages=2, filename='am_ig_posts_2022-01-25') # SH 2023-01-25 23:59 short-lived access token


https://graph.facebook.com/v15.0/17841401256046961/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token=
Response status code:  200
Response status code:  200
Number of posts: 50
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/am_ig_posts_2022-01-25.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/am_ig_posts_2022-01-25.sav


In [135]:
response_json_test_am

{1: {'data': [{'timestamp': '2023-01-25T20:37:00+0000',
    'caption': 'Why I love my muscles \U0001faf6\n\nWhat you see:\n✅ A girl who loves exercise - possibly even obsessed \n✅ A person who is motivated\n✅ A physically strong female\n✅ Someone who eats really well\n✅ Not a big partier \n✅ Has time to train 6 times a week\n\nWhat I see:\n✅ A rock solid mindset \n✅ Someone who is disciplined \n✅ A physically and mentally strong female\n✅ Someone who nourishes and fuels her body\n✅ The most extrovert, social butterfly you’ll ever meet - but doesn’t prioritise partying\n✅ Commits to training 6 times a week\n\nMindset - one of the most important elements to any health and fitness journey. Perspective changes based on the person - we all know the glass half empty/half full example.\n\nIf you’re eating well and doing physical exercise but not prioritising your mental exercise - I can promise you what you’re doing is not sustainable in the long term! \n\nWork on your mindset and the game ch

In [108]:
df_test_am.head()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next
0,2023-01-25T20:37:00+0000,Why I love my muscles 🫶\n\nWhat you see:\n✅ A ...,62,1,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cn2g-nCBzCZ/,17902790093652450,"[{'timestamp': '2023-01-25T20:50:54+0000', 'te...",NaN,NaN,NaN
1,2023-01-24T19:20:12+0000,Full recipe in the video ⬆️\n\nThe one thing t...,50,2,VIDEO,REELS,NaN,https://www.instagram.com/reel/CnzzEKNhdLh/,17976757489963975,"[{'timestamp': '2023-01-24T20:04:16+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
2,2023-01-23T18:52:42+0000,I used to talk a lot about the impact of my me...,17,0,VIDEO,REELS,https://video-sea1-1.cdninstagram.com/o1/v/t16...,https://www.instagram.com/reel/CnxK_ixhiuM/,17976666928950022,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
3,2023-01-22T19:01:08+0000,𝐂𝐫𝐞𝐚𝐭𝐢𝐧𝐞\nCreatine is one of the most vastly r...,14,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cnunn04PpGs/,17918224373661214,NaN,NaN,NaN,NaN
4,2023-01-22T09:15:56+0000,Sunday evening thoughts 💭 \n\nAs you approach ...,72,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cntkpvuhj60/,17994459673604175,NaN,NaN,NaN,NaN


In [109]:
df_test_am.tail()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next
20,2022-12-01T07:04:26+0000,Free December Deal 🔥🔥🔥\n\nYou won’t want to mi...,52,54,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clnb7OHBMdZ/,17972337106905374,"[{'timestamp': '2022-12-02T23:42:22+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,UVZAGRFMxRjNXRFJKTFhBdFdHdFZAlSGM1T1RCTlQyTkxR...,https://graph.facebook.com/v15.0/1797233710690...
21,2022-11-29T18:52:26+0000,Nutrition and Training Workshop 💪\n\nMy ethos ...,18,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cljjs0dhnUX/,17846185016881186,NaN,NaN,NaN,NaN
22,2022-11-28T18:57:08+0000,10 minutes to prepare and cook ✅\n\nServing se...,54,0,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clg_JRyBbRU/,17878226135730844,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
23,2022-11-27T19:20:45+0000,Mission accomplished ✅🎉\n\nTo say I’m proud wo...,165,53,VIDEO,REELS,NaN,https://www.instagram.com/reel/CledPjThEWL/,17965603229022919,"[{'timestamp': '2022-11-28T07:49:29+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
24,2022-11-26T07:22:12+0000,One of the hardest parts of having online clie...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN


## version 2

In [ ]:
# SH 2023-01-26 0:29 See updated version
def get_user_ig_post_text(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ SH 2023-01-25 23:18 Modify so it won't pull duplicate pages
    Pull the media from a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - pages: Number of pages of results to retrieve starting in reverse chronological order.
            Each page of results provides 25 posts.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information (1 post/reel per row)
            - caption
            - like count
            - comments count
            - top-level comments: If the amount of comments is high (>53 commments), it will provide the endpoints, 
                it will return additional endpoints for additional comments ('comments.paging.*').
            - media type: video or image
            - media product type: reels or feed
            - media URL (not available for all reels)
            - permalink
            - timestamp
            - post id
            - comments.data (list of dictionaries): timestamp, username, text, like count, and comment id.
            - thumbnail URL of videos
        - response_json: JSON object with each page number of results as the key (starting with 1)

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2:
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        except:
            print('No data in request response for page', page)
        try:
            # url = response_json_dict[page]['paging']['next']
            next_endpoint = response_json_dict[page]['paging']['next']
            if page > 1 & next_endpoint != url:
                url = next_endpoint
                print(page)
            else:
                print('end')
                break
        except: 
            pass
    try:
        df = pd.concat(df_list)
        print('Number of posts:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict


In [181]:
df_test_am2, response_json_test_am2 = get_user_ig_post_text(credentials['am_ig_user_id'], 
        credentials['am_access_token'], pages=2)


https://graph.facebook.com/v15.0/17841401256046961/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token=
Response status code:  400
{'message': '(#10) Application does not have permission for this action', 'type': 'OAuthException', 'code': 10, 'fbtrace_id': 'AWXXaKeJkWQzRaOwjNQSOcW'}


In [167]:
df_test_am2.url

'https://graph.facebook.com/v15.0/17841401256046961/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token=EAAKEofVXnvEBADUGb992a3P5sjDR5Hfh1uZBksrb34Ou1MDKAMQPNelFtJZBWVW56QmnZBkT1fRHnzEB1b5MZCmoZBTnZAC4IC7yp0krTCUdrq6Px0Bmpw7h4cvFb8J59F4duGo2Nzo9nAaKwoi8CNHv7DZBQqiL15m3YTkw2KFBtBl3rSZAhZBjfYjTMTqG63ZBAZD'

In [174]:
response_json_test_am2
# For some reason, long-lived access token works for pulling AM's FB posts, but not for pulling IG posts.
# Perhaps it's because a separate IG access token is required: https://developers.facebook.com/docs/instagram-basic-display-api/getting-started


{1: {'error': {'message': '(#10) Application does not have permission for this action',
   'type': 'OAuthException',
   'code': 10,
   'fbtrace_id': 'AZzonoLqwQz9-ONwxqplsfq'}}}

In [ ]:
def get_user_ig_post_text(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ SH 2023-01-25 23:18 Modify so it won't pull duplicate pages
    Pull the media from a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - pages: Number of pages of results to retrieve starting in reverse chronological order.
            Each page of results provides 25 posts.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information (1 post/reel per row)
            - caption
            - like count
            - comments count
            - top-level comments: If the amount of comments is high (>53 commments), it will provide the endpoints, 
                it will return additional endpoints for additional comments ('comments.paging.*').
            - media type: video or image
            - media product type: reels or feed
            - media URL (not available for all reels)
            - permalink
            - timestamp
            - post id
            - comments.data (list of dictionaries): timestamp, username, text, like count, and comment id.
            - thumbnail URL of videos
        - response_json: JSON object with each page number of results as the key (starting with 1)

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)

## version 3

In [187]:
def test_ig_credentials(ig_user_id, access_token):
    """ 

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}?fields=username&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response = requests.get(url)
    print('Response status code: ',response.status_code)
    response_json = response.json()
        
    return response_json

test_ig_credentials(credentials['am_ig_user_id'], credentials['am_access_token'])

https://graph.facebook.com/v15.0/17841401256046961?fields=username&access_token=
Response status code:  400


{'error': {'message': '(#10) Application does not have permission for this action',
  'type': 'OAuthException',
  'code': 10,
  'fbtrace_id': 'ArqOsoZX9VwhYcVAXgeWzTT'}}

In [188]:
test_ig_credentials(credentials['ig_user_id'], credentials['access_token'])

https://graph.facebook.com/v15.0/17841403231458630?fields=username&access_token=
Response status code:  200


{'username': 'silvialiftsweights', 'id': '17841403231458630'}

In [189]:
test_ig_credentials(credentials['am_ig_user_id'], credentials['access_token'])

https://graph.facebook.com/v15.0/17841401256046961?fields=username&access_token=
Response status code:  400


{'error': {'message': '(#10) Application does not have permission for this action',
  'type': 'OAuthException',
  'code': 10,
  'fbtrace_id': 'AUsHx0mif0kI_TA6_aqE3xX'}}

In [3]:
# Looks like I deleted the cell where I pulled my own IG posts

## Version 4: Grab AM's posts

In [332]:
def test_ig_credentials(ig_user_id, access_token):
    """ 

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}?fields=username&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response = requests.get(url)
    print('Response status code: ',response.status_code)
    response_json = response.json()
        
    return response_json

test_ig_credentials(credentials['am_ig_user_id'], credentials['am3_access_token'])

https://graph.facebook.com/v15.0/17841401256046961?fields=username&access_token=
Response status code:  200


{'username': 'coach_mcloone', 'id': '17841401256046961'}

In [338]:

def get_user_ig_post_text(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ SH 2023-01-25 23:18 Modify so it won't pull duplicate pages
    Pull the media from a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - pages: Number of pages of results to retrieve starting in reverse chronological order.
            Each page of results provides 25 posts.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information (1 post/reel per row)
            - caption
            - like count
            - comments count
            - top-level comments: If the amount of comments is high (>53 commments), it will provide the endpoints, 
                it will return additional endpoints for additional comments ('comments.paging.*').
            - media type: video or image
            - media product type: reels or feed
            - media URL (not available for all reels)
            - permalink
            - timestamp
            - post id
            - comments.data (list of dictionaries): timestamp, username, text, like count, and comment id.
            - thumbnail URL of videos
        - response_json: JSON object with each page number of results as the key (starting with 1)

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        except:
            print('No data in request response for page', page)
        try:
            next_endpoint = response_json_dict[page]['paging']['next']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            pass
    try:
        df = pd.concat(df_list)
        print('Number of posts:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df_test_am, response_json_test_am = get_user_ig_post_text(credentials['am_ig_user_id'], 
        credentials['am3_access_token'], pages=2)


https://graph.facebook.com/v15.0/17841401256046961/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token=
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Number of posts: 50


In [ ]:
response_json_test_am

In [340]:
df_am50, response_json_am50 = get_user_ig_post_text(credentials['am_ig_user_id'], 
        credentials['am3_access_token'], pages=50, filename='am_ig_posts_50pages_2023-01-26')

https://graph.facebook.com/v15.0/17841401256046961/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token=
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Requesting page 3...
	Response status code:  200
Requesting page 4...
	Response status code:  200
Requesting page 5...
	Response status code:  200
Requesting page 6...
	Response status code:  200
Requesting page 7...
	Response status code:  200
Requesting page 8...
	Response status code:  200
Requesting page 9...
	Response status code:  200
Requesting page 10...
	Response status code:  200
Requesting page 11...
	Response status code:  200
Requesting page 12...
	Response status code:  200
Requesting page 13...
	Response status code:  200
Requesting page 14...
	Response status 

In [345]:
filename='am_ig_posts_50pages_2023-01-26'
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
save_csv(df_am50,filename,csv_path)
savepickle(response_json_am50,filename,'sav',json_path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/am_ig_posts_50pages_2023-01-26.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/am_ig_posts_50pages_2023-01-26.sav


# Load my IG post data

In [247]:
filename='am_ig_posts_2022-01-25.csv'
path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
df_test_am = load_csv(filename, path, column1_as_index=True)

Dataframe shape:  (50, 13)


In [6]:
df_test_am.tail()

,Unnamed: 0,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next
45,20,2022-12-01T07:04:26+0000,Free December Deal 🔥🔥🔥\n\nYou won’t want to mi...,52,54,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clnb7OHBMdZ/,17972337106905374,"[{'timestamp': '2022-12-02T23:42:22+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,UVZAGRFMxRjNXRFJKTFhBdFdHdFZAlSGM1T1RCTlQyTkxR...,https://graph.facebook.com/v15.0/1797233710690...
46,21,2022-11-29T18:52:26+0000,Nutrition and Training Workshop 💪\n\nMy ethos ...,18,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cljjs0dhnUX/,17846185016881186,NaN,NaN,NaN,NaN
47,22,2022-11-28T18:57:08+0000,10 minutes to prepare and cook ✅\n\nServing se...,54,0,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clg_JRyBbRU/,17878226135730844,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
48,23,2022-11-27T19:20:45+0000,Mission accomplished ✅🎉\n\nTo say I’m proud wo...,165,53,VIDEO,REELS,NaN,https://www.instagram.com/reel/CledPjThEWL/,17965603229022919,"[{'timestamp': '2022-11-28T07:49:29+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
49,24,2022-11-26T07:22:12+0000,One of the hardest parts of having online clie...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN


In [307]:
filename='my_ig_posts_50pages_2023-01-26.csv'
path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
df_me50 = load_csv(filename, path, column1_as_index=True)

Dataframe shape:  (730, 11)


# Process post data

## `process_df_timestamp`

In [308]:
import re
def process_df_timestamp(input_df):
    """
    Convert dates in the json-derived dataframe into different formats.
    """
    df = input_df.reset_index(drop=True)
    regex_date = r'.+T'
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['time'] = df['timestamp'].dt.time
    df['hour'] = df['timestamp'].dt.hour
    return df
process_df_timestamp(df_test_am.tail(1).copy())

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next,date,year,month,day_of_week,time,hour
0,2022-11-26 07:22:12+00:00,One of the hardest parts of having online clie...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN,2022-11-26,2022,11,5,07:22:12,7


## process text
### version 1

In [127]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
def preprocess_post_text(docs):
    """
    Prepare data from text documents for NLP:
    - Remove all the special characters
    - Remove all single characters
    - Substitute multiple spaces with single space
    - Convert to lowercase

    Parameters:
    docs (n x 1 array or string): Documents.

    Returns: Array of processed docs.
    """
    clean_docs = []
    wnl = WordNetLemmatizer()
    for doc in docs:
        try:
            # Split text into single words (also gets rid of extra white spaces)
            words = word_tokenize(doc)

            # Convert to lower case
            words = [word.lower() for word in words]

            # Lemmatize words (must be done after conversion to lower case)
            words = [wnl.lemmatize(word) for word in words]

            # Remove all single characters
            words = [word for word in words if len(word) > 2]
            
            # join words back together as a string
            words = ''.join([word+' ' for word in words])

            # Remove any URLs 
            words = re.sub(r'\w*\.+\w*', '', words) # Remove periods in middle of word
            words = re.sub(r'\w*/+\w*', '', words) # remove forward slash in middle of word
            words = re.sub(r'\w*/+\w*', '', words)

            # remove special characters
            words = ''.join([char for char in words if char not in string.punctuation])
            clean_docs.append(words)
        except: # In case value is nan
            clean_docs.append(doc)
        
    return np.array(clean_docs)

from sklearn.feature_extraction.text import CountVectorizer
from custom_nlp import *
def ig_post_preprocessing(df):
    df = process_df_timestamp(df)
    df['caption'] = preprocess_post_text(df['caption'])
    vect = CountVectorizer()
    vect.fit(df['caption'])
    vector = vect.transform(df['caption'])
    print('shape: ', vector.shape)
    vector_df = pd.DataFrame(vector.toarray(), columns=vect.get_feature_names_out())

    return df, vector_df

df, count_vector = ig_post_preprocessing(df_test_am.tail().copy())

shape:  (5, 266)


In [128]:
df

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next,date,year,month,day_of_week,time,hour
0,2022-12-01 07:04:26+00:00,free december deal 🔥🔥🔥 you won want miss want ...,52,54,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clnb7OHBMdZ/,17972337106905374,"[{'timestamp': '2022-12-02T23:42:22+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,UVZAGRFMxRjNXRFJKTFhBdFdHdFZAlSGM1T1RCTlQyTkxR...,https://graph.facebook.com/v15.0/1797233710690...,2022-12-01,2022,12,3,07:04:26,7
1,2022-11-29 18:52:26+00:00,nutrition and training workshop ethos help wom...,18,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cljjs0dhnUX/,17846185016881186,NaN,NaN,NaN,NaN,2022-11-29,2022,11,1,18:52:26,18
2,2022-11-28 18:57:08+00:00,minute prepare and cook serving serf 34 portio...,54,0,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clg_JRyBbRU/,17878226135730844,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN,2022-11-28,2022,11,0,18:57:08,18
3,2022-11-27 19:20:45+00:00,mission accomplished say proud would understat...,165,53,VIDEO,REELS,NaN,https://www.instagram.com/reel/CledPjThEWL/,17965603229022919,"[{'timestamp': '2022-11-28T07:49:29+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN,2022-11-27,2022,11,6,19:20:45,19
4,2022-11-26 07:22:12+00:00,one the hardest part having online client not ...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN,2022-11-26,2022,11,5,07:22:12,7


In [132]:
count_vector.sum().sort_values(ascending=False)

the              18
and              15
you              13
this             10
with              7
                 ..
building          1
let               1
liftwithlaura     1
bring             1
link              1
Length: 266, dtype: int64

SH 2023-01-25 22:54
Next steps:
* Include hashtags
* Create regular expression that replaces all tokens starting with a digit into '[number]'
* Allow optional arguments to be entered into the CountVectorizer instantiation:
    * stop_words

### version 2
Add optional parameters to CountVectorizer

In [204]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [207]:
''.join([char for char in string.punctuation if char not in '#'])

'!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [314]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
def preprocess_post_text(docs):
    """
    Prepare data from text documents for NLP:
    - Remove all the special characters
    - Remove all single characters
    - Substitute multiple spaces with single space
    - Convert to lowercase

    Parameters:
    docs (n x 1 array or string): Documents.

    Returns: Array of processed docs.
    """
    clean_docs = []
    wnl = WordNetLemmatizer()
    no_captions = 0
    for doc in docs:
        try:
            # Split text into single words (also gets rid of extra white spaces)
            words = word_tokenize(doc)

            # Convert to lower case
            words = [word.lower() for word in words]

            # Lemmatize words (must be done after conversion to lower case)
            words = [wnl.lemmatize(word) for word in words]
            
            # join words back together as a string
            words = ''.join([word+' ' for word in words])

            # Join @ and # to the subsequent word to retain handles and hashtags
            words = re.sub(r'@ \w+', 'zzzHandle', words)
            words = re.sub(r'# \w+', 'zzzHashtag', words)
            # words = re.sub(r'@ \w+', '[handle]', words)
            # words = re.sub(r'# \w+', '[hashtag]', words)

            # Remove any URLs 
            words = re.sub(r'\w*\.+\w*', '', words) # Remove periods in middle of word
            words = re.sub(r'\w*/+\w*', '', words) # remove forward slash in middle of word
            words = re.sub(r'\w*/+\w*', '', words)

            # Replace hypthens with spaces
            # words = re.sub(r'(\b\w+)-(\w+\b)', r'\1 \2')
            words = re.sub(r'(\w+)-(\w+)*', r'\1 \2', words)
            words = re.sub(r'(\w+)-(\w+)*', r'\1 \2', words)

            # Remove numbers
            words = re.sub(r'\d:\d\d[\-a-zA-Z]*','zzzTime', words) # Time of day
            words = re.sub(r'\b\d+\b', 'zzzNumber', words)
            words = re.sub(r'\b\d+\w+\b', 'zzzNumber', words) #Number with letters

            # remove special characters
            non_hashtag_punctuation = ''.join([char for char in string.punctuation if char not in '#@'])
            words = ''.join([char for char in words if char not in non_hashtag_punctuation])

            clean_docs.append(words)
        except: # In case value is nan
            no_captions +=1
            # print(doc)
            clean_docs.append('zzzEmpty')
    print(f'Unable to completely preprocess {no_captions} captions')
    print('Time processed:', datetime.now())
    return np.array(clean_docs)

from sklearn.feature_extraction.text import CountVectorizer
from custom_nlp import *
def post_preprocessing(input_df, text_column='caption', n_top_to_print=10, **kwargs):
    df = process_df_timestamp(input_df)
    df[text_column] = preprocess_post_text(df[text_column])

    # kwargs
    stop_words = kwargs.get('stop_words', 'english')
    token_pattern = kwargs.get('token_pattern', r"(?u)\b\w\w+\b")
    ngram_range = kwargs.get('ngram_range', (1,1))
    max_df = kwargs.get('max_df', 1.0)
    min_df = kwargs.get('min_df', 2)
    max_features = kwargs.get('max_features', None)
    
    print('Token pattern:', token_pattern)

    vect = CountVectorizer(
            stop_words=stop_words, 
            token_pattern=token_pattern,
            ngram_range=ngram_range,
            max_df=max_df,
            min_df=min_df,
            max_features=max_features
        )

    vect.fit(df['caption'])
    vector = vect.transform(df['caption'])
    print('Shape of vector array: ', vector.shape)
    vector_df = pd.DataFrame(vector.toarray(), columns=vect.get_feature_names_out())

    # Replace zzz tags with brackets 
    vector_df.columns = vector_df.columns.str.replace(r'zzz(\w+)', r'<\1>', regex=True)
    print(f'\nTop {n_top_to_print} words:')
    print(vector_df.sum().sort_values(ascending=False).head(n_top_to_print))

    return df, vector_df

token_pattern = r'(?u)\b[a-zA-Z][a-zA-Z]+\b'
df, count_vector = post_preprocessing(df_test_am.tail().copy(), 
                    # token_pattern=token_pattern,
                    # max_features=3
                    )
# print(count_vector.sum().sort_values(ascending=False))
# print(count_vector.sum().sort_index())
count_vector

Unable to completely preprocess 0 captions
Time processed: 2023-01-26 21:23:34.913730
Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (5, 19)

Top 10 words:
<hashtag>    17
<number>     16
goal          4
like          4
going         3
moment        2
woman         2
training      2
time          2
start         2
dtype: int64


,able,best,biggest,client,community,goal,going,learn,like,ll,minded,moment,proud,start,time,training,woman,<hashtag>,<number>
0,0,1,0,1,0,2,0,0,0,1,0,0,0,1,0,0,0,11,5
1,0,0,1,0,1,1,1,1,2,1,1,0,0,0,1,1,1,0,3
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3
3,1,1,1,0,0,0,2,1,0,0,0,1,1,1,0,1,0,0,5
4,1,0,0,1,1,1,0,0,2,0,1,1,1,0,1,0,1,0,0


In [290]:
count_vector.sum().sort_values(ascending=False).head()

<hashtag>    17
<number>     16
goal          4
like          4
going         3
dtype: int64

In [221]:
docs = ['test-test this is a test-test-test', 'hello-me silvia-is-the-best']
preprocess_post_text(docs)

Unable to completely preprocess 0 captions


array(['test test this is a test test test ',
       'hello me silvia is the best '], dtype='<U35')

In [270]:
count_vector.filter(regex='<.+>')

,<hashtag>,<number>
0,11,5
1,0,3
2,6,3
3,0,5
4,0,0


In [271]:
[a for a in count_vector.columns]

['able',
 'best',
 'biggest',
 'client',
 'community',
 'goal',
 'going',
 'learn',
 'like',
 'll',
 'minded',
 'moment',
 'proud',
 'start',
 'time',
 'training',
 'woman',
 '<hashtag>',
 '<number>']

#### Process my posts

In [315]:
df_me_processed, count_vector_me = post_preprocessing(df_me50, max_features=1000)
count_vector_me

Unable to completely preprocess 46 captions
Time processed: 2023-01-26 21:23:45.366275
Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (730, 1000)

Top 10 words:
<hashtag>    5821
<number>     1397
help          390
time          375
exercise      359
want          311
body          293
feel          266
<handle>      258
work          249
dtype: int64


,ab,abdominal,ability,able,according,accountability,ache,achieve,achy,action,...,yes,yesterday,yogurt,youtube,yummy,zucchini,<empty>,<handle>,<hashtag>,<number>
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,5,0
726,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,2
728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [311]:
df_me50.head()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url
0,2023-01-20T19:38:19+0000,"If you've been to a Lululemon store recently, ...",0,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnpiSZEP982/,18324686224070369,NaN,NaN
1,2023-01-18T18:32:47+0000,"What’s your favourite fitness tech, app, or se...",0,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnkRMwXPDRX/,18269437663142607,NaN,NaN
2,2023-01-13T19:52:47+0000,It’s been 1 month since I finished my data sci...,42,3,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnXiYNmverQ/,18000771625581319,"[{'timestamp': '2023-01-14T15:18:44+0000', 'te...",NaN
3,2023-01-11T03:22:56+0000,There was a time when my only exercise was run...,1,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnQngpIrOIS/,17862580493850465,NaN,NaN
4,2023-01-09T21:38:41+0000,"For the US, ""Wearable technology (#1), strengt...",2,1,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnNbUXKynWR/,18072187321337751,"[{'timestamp': '2023-01-22T02:19:21+0000', 'te...",NaN


In [316]:
[word for word in count_vector_me.columns]

['ab',
 'abdominal',
 'ability',
 'able',
 'according',
 'accountability',
 'ache',
 'achieve',
 'achy',
 'action',
 'activate',
 'activation',
 'active',
 'activity',
 'actually',
 'add',
 'added',
 'adding',
 'addition',
 'address',
 'advice',
 'affect',
 'age',
 'ago',
 'aide',
 'aim',
 'aligned',
 'allow',
 'allows',
 'almond',
 'amazing',
 'ankle',
 'answer',
 'anterior',
 'app',
 'applies',
 'approach',
 'apr',
 'arch',
 'area',
 'arm',
 'article',
 'asleep',
 'association',
 'athlete',
 'attempt',
 'available',
 'avocado',
 'avoid',
 'avoiding',
 'away',
 'awesome',
 'baby',
 'bacteria',
 'bad',
 'bag',
 'bake',
 'baked',
 'baking',
 'balance',
 'balancing',
 'ball',
 'banana',
 'band',
 'bar',
 'barbell',
 'barrier',
 'based',
 'batch',
 'beef',
 'behaviour',
 'believe',
 'belly',
 'bend',
 'bending',
 'benefit',
 'best',
 'better',
 'big',
 'biggest',
 'bike',
 'bio',
 'bit',
 'blade',
 'blend',
 'blender',
 'blood',
 'body',
 'bodyweight',
 'bone',
 'bonus',
 'book',
 'boring

In [ ]:
index = count_vector_me[count_vector_me['<numberzzznumber>'] > 0].index
print(index)
for i, text in df_me_processed.loc[index,'caption'].items():
    print(text)
    print()

In [229]:
count_vector_me.sum(axis=1).sort_values()

235      0
174      0
561      0
28       0
9        0
      ... 
232    154
629    159
501    160
395    160
233    161
Length: 730, dtype: int64

In [230]:
df_me_processed

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,date,year,month,day_of_week,time,hour
0,2023-01-20 19:38:19+00:00,if you ve been to a lululemon store recently y...,0,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnpiSZEP982/,18324686224070369,NaN,NaN,2023-01-20,2023,1,4,19:38:19,19
1,2023-01-18 18:32:47+00:00,what ’ s your favourite fitness tech app or se...,0,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnkRMwXPDRX/,18269437663142607,NaN,NaN,2023-01-18,2023,1,2,18:32:47,18
2,2023-01-13 19:52:47+00:00,it ’ s been znumber month since i finished my ...,42,3,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnXiYNmverQ/,18000771625581319,"[{'timestamp': '2023-01-14T15:18:44+0000', 'te...",NaN,2023-01-13,2023,1,4,19:52:47,19
3,2023-01-11 03:22:56+00:00,there wa a time when my only exercise wa runni...,1,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnQngpIrOIS/,17862580493850465,NaN,NaN,2023-01-11,2023,1,2,03:22:56,3
4,2023-01-09 21:38:41+00:00,for the u wearable technology zhashtag strengt...,2,1,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CnNbUXKynWR/,18072187321337751,"[{'timestamp': '2023-01-22T02:19:21+0000', 'te...",NaN,2023-01-09,2023,1,0,21:38:41,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2017-03-08 01:10:05+00:00,green soup smoothy with homemade bone broth bo...,10,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/BRW7JiWlf3C/,17863217233097922,NaN,NaN,2017-03-08,2017,3,2,01:10:05,1
726,2017-03-05 23:40:02+00:00,zhashtag zhashtag zhashtag zhashtag,5,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/BRRnQJ0lHQp/,17875151227038539,NaN,NaN,2017-03-05,2017,3,6,23:40:02,23
727,2017-03-05 18:56:28+00:00,baked protein pancake approx znumber kcal and ...,27,4,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/BRRGzPIFmwF/,17877397126032942,"[{'timestamp': '2017-04-05T04:53:44+0000', 'te...",NaN,2017-03-05,2017,3,6,18:56:28,18
728,2017-03-05 18:43:52+00:00,zhashtag zhashtag,6,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/BRRFXBpl2iG/,17850956851157748,NaN,NaN,2017-03-05,2017,3,6,18:43:52,18


### Version 3
Reduce for loops for faster processing

In [323]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
def preprocess_post_text(doc):
    """
    Prepare data from text documents for NLP:
    - Remove all the special characters
    - Remove all single characters
    - Substitute multiple spaces with single space
    - Convert to lowercase

    Parameters:
    doc (string): Document.

    Returns: Processed doc.
    """
    wnl = WordNetLemmatizer()
    try:
        # Split text into single words (also gets rid of extra white spaces)
        words = word_tokenize(doc)

        # Convert to lower case
        words = [word.lower() for word in words]

        # Lemmatize words (must be done after conversion to lower case)
        words = [wnl.lemmatize(word) for word in words]
        
        # join words back together as a string
        words = ''.join([word+' ' for word in words])

        # Join @ and # to the subsequent word to retain handles and hashtags
        words = re.sub(r'@ \w+', 'zzzHandle', words)
        words = re.sub(r'# \w+', 'zzzHashtag', words)

        # Remove any URLs 
        words = re.sub(r'\w*\.+\w*', '', words) # Remove periods in middle of word
        words = re.sub(r'\w*/+\w*', '', words) # remove forward slash in middle of word
        words = re.sub(r'\w*/+\w*', '', words)

        # Replace hypthens with spaces
        words = re.sub(r'(\w+)-(\w+)*', r'\1 \2', words)
        words = re.sub(r'(\w+)-(\w+)*', r'\1 \2', words)

        # Remove numbers
        words = re.sub(r'\d:\d\d[\-a-zA-Z]*','zzzTime', words) # Time of day
        words = re.sub(r'\b\d+\b', 'zzzNumber', words)
        words = re.sub(r'\b\d+\w+\b', 'zzzNumber', words) #Number with letters

        # remove special characters
        non_hashtag_punctuation = ''.join([char for char in string.punctuation if char not in '#@'])
        words = ''.join([char for char in words if char not in non_hashtag_punctuation])

        return words
    except: # In case value is nan
        return 'zzzEmpty'

from sklearn.feature_extraction.text import CountVectorizer
from custom_nlp import *
def post_preprocessing(input_df, text_column='caption', n_top_to_print=10, **kwargs):
    df = process_df_timestamp(input_df)
    df[text_column] = df[text_column].apply(lambda x: preprocess_post_text(x))

    # kwargs
    stop_words = kwargs.get('stop_words', 'english')
    token_pattern = kwargs.get('token_pattern', r"(?u)\b\w\w+\b")
    ngram_range = kwargs.get('ngram_range', (1,1))
    max_df = kwargs.get('max_df', 1.0)
    min_df = kwargs.get('min_df', 2)
    max_features = kwargs.get('max_features', None)
    
    print('Token pattern:', token_pattern)

    vect = CountVectorizer(
            stop_words=stop_words, 
            token_pattern=token_pattern,
            ngram_range=ngram_range,
            max_df=max_df,
            min_df=min_df,
            max_features=max_features
        )

    vect.fit(df['caption'])
    vector = vect.transform(df['caption'])
    print('Shape of vector array: ', vector.shape)
    vector_df = pd.DataFrame(vector.toarray(), columns=vect.get_feature_names_out())

    # Replace zzz tags with brackets 
    vector_df.columns = vector_df.columns.str.replace(r'zzz(\w+)', r'<\1>', regex=True)
    print(f'\nTop {n_top_to_print} words:')
    print(vector_df.sum().sort_values(ascending=False).head(n_top_to_print))
    print('Time processed:', datetime.now())

    return df, vector_df

token_pattern = r'(?u)\b[a-zA-Z][a-zA-Z]+\b'
df, count_vector = post_preprocessing(df_test_am.tail().copy(), 
                    # token_pattern=token_pattern,
                    # max_features=3
                    )
# print(count_vector.sum().sort_values(ascending=False))
# print(count_vector.sum().sort_index())
count_vector

Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (5, 19)

Top 10 words:
<hashtag>    17
<number>     16
goal          4
like          4
going         3
moment        2
woman         2
training      2
time          2
start         2
dtype: int64
Time processed: 2023-01-26 21:37:10.204877


,able,best,biggest,client,community,goal,going,learn,like,ll,minded,moment,proud,start,time,training,woman,<hashtag>,<number>
0,0,1,0,1,0,2,0,0,0,1,0,0,0,1,0,0,0,11,5
1,0,0,1,0,1,1,1,1,2,1,1,0,0,0,1,1,1,0,3
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3
3,1,1,1,0,0,0,2,1,0,0,0,1,1,1,0,1,0,0,5
4,1,0,0,1,1,1,0,0,2,0,1,1,1,0,1,0,1,0,0


#### Process my posts

In [324]:
df_me_processed, count_vector_me = post_preprocessing(df_me50, max_features=1000)
count_vector_me

Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (730, 1000)

Top 10 words:
<hashtag>    5821
<number>     1397
help          390
time          375
exercise      359
want          311
body          293
feel          266
<handle>      258
work          249
dtype: int64
Time processed: 2023-01-26 21:39:09.590677


,ab,abdominal,ability,able,according,accountability,ache,achieve,achy,action,...,yes,yesterday,yogurt,youtube,yummy,zucchini,<empty>,<handle>,<hashtag>,<number>
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,5,0
726,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,2
728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [325]:
[word for word in count_vector_me.columns]

['ab',
 'abdominal',
 'ability',
 'able',
 'according',
 'accountability',
 'ache',
 'achieve',
 'achy',
 'action',
 'activate',
 'activation',
 'active',
 'activity',
 'actually',
 'add',
 'added',
 'adding',
 'addition',
 'address',
 'advice',
 'affect',
 'age',
 'ago',
 'aide',
 'aim',
 'aligned',
 'allow',
 'allows',
 'almond',
 'amazing',
 'ankle',
 'answer',
 'anterior',
 'app',
 'applies',
 'approach',
 'apr',
 'arch',
 'area',
 'arm',
 'article',
 'asleep',
 'association',
 'athlete',
 'attempt',
 'available',
 'avocado',
 'avoid',
 'avoiding',
 'away',
 'awesome',
 'baby',
 'bacteria',
 'bad',
 'bag',
 'bake',
 'baked',
 'baking',
 'balance',
 'balancing',
 'ball',
 'banana',
 'band',
 'bar',
 'barbell',
 'barrier',
 'based',
 'batch',
 'beef',
 'behaviour',
 'believe',
 'belly',
 'bend',
 'bending',
 'benefit',
 'best',
 'better',
 'big',
 'biggest',
 'bike',
 'bio',
 'bit',
 'blade',
 'blend',
 'blender',
 'blood',
 'body',
 'bodyweight',
 'bone',
 'bonus',
 'book',
 'boring

In [327]:
count_vector_me['<empty>'].sum()

46

# *End of Page*